**Customer Segmentation**

Customer Segmentation can be done on different types

1.  Location Based
2. Demographic Based
3. Psycho-graphic Based
4. Behavioral Based

Dataset doesn't have enough information(features) to perform all the segmentation. 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
data = pd.read_csv('../input/data.csv', encoding = 'cp1252')

In [ ]:
data.shape

Dataset contains 541909 rows and 8 columns

Generates descriptive statistics that summarize the central tendency, dispersion and shape of a dataset’s distribution

In [ ]:
data.describe()

Dataset has negative values in Quantity and UnitPrice. remove those entries from dataset

In [ ]:
df = data[(data.Quantity > 0)]
df = df[(df.UnitPrice > 0)]

In [ ]:
df.describe()

Plot data in Quantity andUnitPrice

In [ ]:
plot = df.plot.scatter('UnitPrice', 'Quantity')

Based on the above graph we could see some of ouliers in the dataset

In [ ]:
df = df[(df.Quantity < 500)] 
df = df[(df.UnitPrice < 100)]

In [ ]:
#Check Null values are present in the dataset
df.isnull().sum()

Customer column has 131591 null values. Customer Id is unique values since we cant replace with any other values, hence remove it from the dataset

In [ ]:
df=df.dropna(how = 'any', axis = 'rows')
df.shape

In [ ]:
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

In [ ]:
df.head()

Number of Orders by Country

In [ ]:
ord_cnty = df.groupby('Country')['InvoiceNo'].count().sort_values(ascending=False)

#Plot
plt.figure(figsize=(12,6))
ord_cnty.plot.bar()
plt.xlabel('Number of Orders')
plt.ylabel('Country')
plt.title('Number of Orders per Country', fontsize=16)
plt.show()

Amount spend by Country

In [ ]:
amt_cnty = df.groupby('Country')['TotalPrice'].sum().sort_values(ascending=False)

#Plot
plt.figure(figsize=(12,6))
amt_cnty.plot.bar()
plt.xlabel('Country')
plt.ylabel('TotalPrice')
plt.title('Amount Spend per Country', fontsize=16)
plt.show()

Group data based on Customer ID

In [ ]:
dfg = df.groupby(by=['CustomerID'])['Quantity','TotalPrice'].agg({'Quantity':'sum','TotalPrice':'sum'}).reset_index()
dfg.rename(columns={'Quantity':'Total_Quantity','TotalPrice' : 'Total_Amount'},inplace=True)
dfg.head(3)

In [ ]:
X = dfg.iloc[:, [1, 2]].values
# Feature Scaling
sc_X = StandardScaler()
X= sc_X.fit_transform(X)

In [ ]:
wcss=[]
for n_clusters in range(1,10):
    kmeans=KMeans(n_clusters=n_clusters,init='k-means++',)
    #KModes(n_clusters=4, init='Huang', n_init=5, verbose=1)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
    clusters = kmeans.predict(X)
plt.plot(range(1,10),wcss)
plt.title('Elbow Method')
plt.xlabel('No. of cluster')
plt.ylabel('wcss: sum of dist. of sample to their closest cluster center' )

In [ ]:
# Fitting K-Means to the dataset
kmeans = KMeans(n_clusters = 3, init = 'k-means++')
y_kmeans = kmeans.fit_predict(X)

In [ ]:
plt.scatter(X[y_kmeans == 0, 0], X[y_kmeans == 0, 1], s = 50, c = 'red', label = 'Customer Type 1')
plt.scatter(X[y_kmeans == 1, 0], X[y_kmeans == 1, 1], s = 50, c = 'blue', label = 'Customer Type 2')
plt.scatter(X[y_kmeans == 2, 0], X[y_kmeans == 2, 1], s = 50, c = 'green', label = 'Customer Type 3')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s = 100, c = 'yellow', label = 'Centroids')
plt.title('Type Of Customers(customer type clusters)')
plt.xlabel('Number of items Purchased(Quantity)')
plt.ylabel('Product price per unit in sterling(Unit Price)')
plt.legend()
plt.show()